In [1]:
# You can start coding here
from IPython.display import IFrame
link_id = "wx7tu"

In [2]:
# imports
import time

tic = time.time()

import pandas as pd
import intake
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import xesmf as xe
import pyleoclim as pyleo

from xmip.preprocessing import combined_preprocessing
from xarrayutils.plotting import shaded_line_plot
import geoviews as gv
import holoviews
from geoviews import Dataset as gvDataset
import geoviews.feature as gf
from geoviews import Image as gvImage

from datatree import DataTree
from xmip.postprocessing import _parse_metric
import cartopy as cart
import cartopy.crs as ccrs
import random
import pooch
import os
import tempfile

import geopy
import geopy.distance



In [3]:
# helper functions

def pooch_load(filelocation=None,filename=None,processor=None):
    shared_location='/home/jovyan/shared/Data/Projects/Heatwaves' # this is different for each day
    user_temp_cache=tempfile.gettempdir()
    
    if os.path.exists(os.path.join(shared_location,filename)):
        file = os.path.join(shared_location,filename)
    else:
        file = pooch.retrieve(filelocation,known_hash=None,fname=os.path.join(user_temp_cache,filename),processor=processor)

    return file

In [4]:
# loading CMIP data

col = intake.open_esm_datastore(
    "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
)  # open an intake catalog containing the Pangeo CMIP cloud data

# pick our five example models
# There are many more to test out! Try executing `col.df['source_id'].unique()` to get a list of all available models
source_ids = ['MPI-ESM1-2-LR']#['MRI-ESM2-0']
variable_ids = ["tas"]#["tas", "pr"]
# sorted(col.df["source_id"].unique())

In [5]:
# from the full `col` object, create a subset using facet search
cat = col.search(
    source_id=source_ids,
    variable_id= variable_ids,
    member_id="r1i1p1f1",
    table_id="3hr",
    grid_label="gn",
    experiment_id=["historical"],  # add scenarios if interested in projection
    require_all_on=[
        "source_id"
    ],  # make sure that we only get models which have all of the above experiments
)

# convert the sub-catalog into a datatree object, by opening each dataset into an xarray.Dataset (without loading the data)
kwargs = dict(
    preprocess=combined_preprocessing,  # apply xMIP fixes to each dataset
    xarray_open_kwargs=dict(
        use_cftime=True
    ),  # ensure all datasets use the same time index
    storage_options={
        "token": "anon"
    },  # anonymous/public authentication to google cloud storage
)

cat.esmcat.aggregation_control.groupby_attrs = ["source_id", "experiment_id"]
dt = cat.to_datatree(**kwargs)
dt


--> The keys in the returned dictionary of datasets are constructed as follows:
	'source_id/experiment_id'


DataTree('None', parent=None)
└── DataTree('MPI-ESM1-2-LR')
    └── DataTree('historical')
            Dimensions:         (member_id: 1, dcpp_init_year: 1, time: 482120, y: 96,
                                 x: 192, bnds: 2, vertex: 4)
            Coordinates:
                height          float64 2.0
              * y               (y) float64 -88.57 -86.72 -84.86 -83.0 ... 84.86 86.72 88.57
              * x               (x) float64 0.0 1.875 3.75 5.625 ... 352.5 354.4 356.2 358.1
              * time            (time) object 1850-01-01 03:00:00 ... 2015-01-01 00:00:00
                lat_bounds      (y, bnds, x) float64 dask.array<chunksize=(96, 2, 192), meta=np.ndarray>
                lon_bounds      (x, bnds, y) float64 dask.array<chunksize=(192, 2, 96), meta=np.ndarray>
                lon             (x, y) float64 360.0 360.0 360.0 360.0 ... 358.1 358.1 358.1
                lat             (x, y) float64 -88.57 -86.72 -84.86 ... 84.86 86.72 88.57
                lon_verticies   (vertex, x, y) float64 dask.array<chunksize=(1, 192, 96), meta=np.ndarray>
                lat_verticies   (vertex, x, y) float64 dask.array<chunksize=(1, 192, 96), meta=np.ndarray>
              * member_id       (member_id) object 'r1i1p1f1'
              * dcpp_init_year  (dcpp_init_year) float64 nan
            Dimensions without coordinates: bnds, vertex
            Data variables:
                tas             (member_id, dcpp_init_year, time, y, x) float32 dask.array<chunksize=(1, 1, 1456, 96, 192), meta=np.ndarray>
            Attributes: (12/63)
                Conventions:                      CF-1.7 CMIP-6.2
                activity_id:                      CMIP
                branch_method:                    standard
                branch_time_in_child:             0.0
                branch_time_in_parent:            0.0
                cmor_version:                     3.5.0
                ...                               ...
                intake_esm_attrs:variable_id:     tas
                intake_esm_attrs:grid_label:      gn
                intake_esm_attrs:zstore:          gs://cmip6/CMIP6/CMIP/MPI-M/MPI-ESM1-2-...
                intake_esm_attrs:version:         20190710
                intake_esm_attrs:_data_format_:   zarr
                intake_esm_dataset_key:           MPI-ESM1-2-LR/historical

In [6]:
# select just a single model and experiment
tas_historical = dt[source_ids[0]]["historical"].ds.tas
print("The time range is:")
print(
    tas_historical.time[0].data.astype("M8[h]"),
    "to",
    tas_historical.time[-1].data.astype("M8[h]"),
)

The time range is:
1850-01-01T03 to 2015-01-01T00


In [7]:
w_day = 10
area = 200 # km^2, minimum area over which tas>P95 for heatwave to exist
X = 5 # days, minimum number of days for which tas(area)>P95 for heatwave to exist
year_start = "1979-01-01"
year_end = "2020-12-31"

# Compute daily mean from 
tas_selected = tas_historical.sel(time=slice(year_start, year_end))
tas_daily_mean = tas_selected.resample(time='D').mean(dim='time')
tas_rolling_P95 = tas_daily_mean.rolling(time=w_day, center=True).construct("tmp").quantile(.95, dim='tmp')
tas_grtr_P95 = tas_daily_mean>tas_rolling_P95  # the boolean mask
tas_grtr_P95
# tas_daily_mean

<xarray.DataArray 'tas' (member_id: 1, dcpp_init_year: 1, time: 13150, y: 96,
                         x: 192)>
dask.array<gt, shape=(1, 1, 13150, 96, 192), dtype=bool, chunksize=(1, 1, 204, 96, 192), chunktype=numpy.ndarray>
Coordinates:
    height          float64 2.0
  * y               (y) float64 -88.57 -86.72 -84.86 -83.0 ... 84.86 86.72 88.57
  * x               (x) float64 0.0 1.875 3.75 5.625 ... 352.5 354.4 356.2 358.1
    lon             (x, y) float64 360.0 360.0 360.0 360.0 ... 358.1 358.1 358.1
    lat             (x, y) float64 -88.57 -86.72 -84.86 ... 84.86 86.72 88.57
  * member_id       (member_id) object 'r1i1p1f1'
  * dcpp_init_year  (dcpp_init_year) float64 nan
  * time            (time) object 1979-01-01 00:00:00 ... 2015-01-01 00:00:00
    quantile        float64 0.95

In [8]:
tas_grtr_P95 = tas_daily_mean>tas_rolling_P95  # the boolean mask
tas_grtr_P95
# tas_daily_mean

<xarray.DataArray 'tas' (member_id: 1, dcpp_init_year: 1, time: 13150, y: 96,
                         x: 192)>
dask.array<gt, shape=(1, 1, 13150, 96, 192), dtype=bool, chunksize=(1, 1, 204, 96, 192), chunktype=numpy.ndarray>
Coordinates:
    height          float64 2.0
  * y               (y) float64 -88.57 -86.72 -84.86 -83.0 ... 84.86 86.72 88.57
  * x               (x) float64 0.0 1.875 3.75 5.625 ... 352.5 354.4 356.2 358.1
    lon             (x, y) float64 360.0 360.0 360.0 360.0 ... 358.1 358.1 358.1
    lat             (x, y) float64 -88.57 -86.72 -84.86 ... 84.86 86.72 88.57
  * member_id       (member_id) object 'r1i1p1f1'
  * dcpp_init_year  (dcpp_init_year) float64 nan
  * time            (time) object 1979-01-01 00:00:00 ... 2015-01-01 00:00:00
    quantile        float64 0.95

In [9]:
tas_daily_mean

<xarray.DataArray 'tas' (member_id: 1, dcpp_init_year: 1, time: 13150, y: 96,
                         x: 192)>
dask.array<transpose, shape=(1, 1, 13150, 96, 192), dtype=float32, chunksize=(1, 1, 204, 96, 192), chunktype=numpy.ndarray>
Coordinates:
    height          float64 2.0
  * y               (y) float64 -88.57 -86.72 -84.86 -83.0 ... 84.86 86.72 88.57
  * x               (x) float64 0.0 1.875 3.75 5.625 ... 352.5 354.4 356.2 358.1
    lon             (x, y) float64 360.0 360.0 360.0 360.0 ... 358.1 358.1 358.1
    lat             (x, y) float64 -88.57 -86.72 -84.86 ... 84.86 86.72 88.57
  * member_id       (member_id) object 'r1i1p1f1'
  * dcpp_init_year  (dcpp_init_year) float64 nan
  * time            (time) object 1979-01-01 00:00:00 ... 2015-01-01 00:00:00
Attributes:
    cell_measures:  area: areacella
    cell_methods:   area: mean time: point
    comment:        near-surface (usually, 2 meter) air temperature
    history:        2019-09-11T14:13:17Z altered by CMOR: Treated scalar dime...
    long_name:      Near-Surface Air Temperature
    standard_name:  air_temperature
    units:          K

In [10]:
# Define the slice of x and y coordinates you want to select
x_slice = slice(25, 65)  # Longitude Range: Approximately 25°E to 65°E
y_slice = slice(20, 40)   # Latitude Range: Approximately 20°N to 40°N

# selecting a smaller region
tas_daily_mean_ME = tas_daily_mean.sel(x=x_slice, y=y_slice)
tas_grtr_P95_ME = tas_grtr_P95.sel(x=x_slice, y=y_slice)  





In [11]:
tas_daily_mean_ME.where(tas_grtr_P95_ME) 

<xarray.DataArray 'tas' (member_id: 1, dcpp_init_year: 1, time: 13150, y: 10,
                         x: 21)>
dask.array<where, shape=(1, 1, 13150, 10, 21), dtype=float32, chunksize=(1, 1, 204, 10, 21), chunktype=numpy.ndarray>
Coordinates:
    height          float64 2.0
  * y               (y) float64 21.45 23.32 25.18 27.05 ... 34.51 36.37 38.24
  * x               (x) float64 26.25 28.12 30.0 31.88 ... 60.0 61.88 63.75
    lon             (x, y) float64 26.25 26.25 26.25 26.25 ... 63.75 63.75 63.75
    lat             (x, y) float64 21.45 23.32 25.18 27.05 ... 34.51 36.37 38.24
  * member_id       (member_id) object 'r1i1p1f1'
  * dcpp_init_year  (dcpp_init_year) float64 nan
  * time            (time) object 1979-01-01 00:00:00 ... 2015-01-01 00:00:00
    quantile        float64 0.95
Attributes:
    cell_measures:  area: areacella
    cell_methods:   area: mean time: point
    comment:        near-surface (usually, 2 meter) air temperature
    history:        2019-09-11T14:13:17Z altered by CMOR: Treated scalar dime...
    long_name:      Near-Surface Air Temperature
    standard_name:  air_temperature
    units:          K

In [12]:
# apply boolean mask
tas_daily_mean_masked = tas_daily_mean_ME.where(tas_grtr_P95_ME) #.sel(time="2000-07-15").squeeze()

tas_daily_mean_masked_day = tas_daily_mean_masked.sel(time="1979-01-01").squeeze()
tas_grtr_P95_ME_day = tas_grtr_P95_ME.sel(time="1979-01-01").squeeze()



In [13]:
tas_daily_mean_masked_day

<xarray.DataArray 'tas' (y: 10, x: 21)>
dask.array<getitem, shape=(10, 21), dtype=float32, chunksize=(10, 21), chunktype=numpy.ndarray>
Coordinates:
    height          float64 2.0
  * y               (y) float64 21.45 23.32 25.18 27.05 ... 34.51 36.37 38.24
  * x               (x) float64 26.25 28.12 30.0 31.88 ... 60.0 61.88 63.75
    lon             (x, y) float64 26.25 26.25 26.25 26.25 ... 63.75 63.75 63.75
    lat             (x, y) float64 21.45 23.32 25.18 27.05 ... 34.51 36.37 38.24
    member_id       <U8 'r1i1p1f1'
    dcpp_init_year  float64 nan
    time            object 1979-01-01 00:00:00
    quantile        float64 0.95
Attributes:
    cell_measures:  area: areacella
    cell_methods:   area: mean time: point
    comment:        near-surface (usually, 2 meter) air temperature
    history:        2019-09-11T14:13:17Z altered by CMOR: Treated scalar dime...
    long_name:      Near-Surface Air Temperature
    standard_name:  air_temperature
    units:          K

In [14]:
# The mask
tas_grtr_P95_ME_day

<xarray.DataArray 'tas' (y: 10, x: 21)>
dask.array<getitem, shape=(10, 21), dtype=bool, chunksize=(10, 21), chunktype=numpy.ndarray>
Coordinates:
    height          float64 2.0
  * y               (y) float64 21.45 23.32 25.18 27.05 ... 34.51 36.37 38.24
  * x               (x) float64 26.25 28.12 30.0 31.88 ... 60.0 61.88 63.75
    lon             (x, y) float64 26.25 26.25 26.25 26.25 ... 63.75 63.75 63.75
    lat             (x, y) float64 21.45 23.32 25.18 27.05 ... 34.51 36.37 38.24
    member_id       <U8 'r1i1p1f1'
    dcpp_init_year  float64 nan
    time            object 1979-01-01 00:00:00
    quantile        float64 0.95

## Count True values in the mask (tas_grtr_P95_ME_day)

In [15]:
start = time.perf_counter()

test=tas_grtr_P95_ME_day.sum().squeeze().copy()
test.to_numpy()


end = time.perf_counter()
print((end-start))

307.77135862596333


In [16]:
#count_true.values

## Ohad's Code

In [30]:
w_day = 10
area = 200 # km^2, minimum area over which tas>P95 for heatwave to exist
X = 5 # days, minimum number of days for which tas(area)>P95 for heatwave to exist
year_start = "2014-01-01"
year_end = "2015-12-31"

#Compute daily mean from
tas_selected = tas_historical.squeeze().sel(time=slice(year_start, year_end))
tas_daily_mean = tas_selected.resample(time='D').mean(dim='time')
tas_rolling_P95 = tas_daily_mean.chunk(dict(time=-1)).quantile(.95, dim='time')

In [31]:
tas_grtr_P95 = tas_daily_mean>tas_rolling_P95  # the boolean mask
tas_grtr_P95

<xarray.DataArray 'tas' (time: 366, y: 96, x: 192)>
dask.array<gt, shape=(366, 96, 192), dtype=bool, chunksize=(342, 96, 192), chunktype=numpy.ndarray>
Coordinates:
    height          float64 2.0
  * y               (y) float64 -88.57 -86.72 -84.86 -83.0 ... 84.86 86.72 88.57
  * x               (x) float64 0.0 1.875 3.75 5.625 ... 352.5 354.4 356.2 358.1
    lon             (x, y) float64 360.0 360.0 360.0 360.0 ... 358.1 358.1 358.1
    lat             (x, y) float64 -88.57 -86.72 -84.86 ... 84.86 86.72 88.57
    member_id       <U8 'r1i1p1f1'
    dcpp_init_year  float64 nan
  * time            (time) object 2014-01-01 00:00:00 ... 2015-01-01 00:00:00
    quantile        float64 0.95

In [32]:
import time
start = time.perf_counter()
#tas_grtr_P95.values
mask = tas_grtr_P95.as_numpy()#

end = time.perf_counter()
print((end-start))

16.30202464701142


In [33]:
mask_np = mask.to_numpy()

In [34]:
start = time.perf_counter()

mask_1d = tas_grtr_P95.sel(time="2014-01-01").squeeze().as_numpy()

end = time.perf_counter()
print((end-start))

mask_1d.to_numpy()

16.111002595047466


array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [ ]:
tas_grtr_P95.sum()

In [ ]:
start = time.perf_counter()

count_true = tas_grtr_P95.sum().to_numpy()

end = time.perf_counter()
print((end-start))
count_true

In [ ]:
tas_grtr_P95.sum()

In [ ]:
start = time.perf_counter()

x = tas_rolling_P95.as_numpy()

end = time.perf_counter()
print((end-start))
x.to_numpy()